# Agentic Sales Pipeline

## Initial Imports

In [20]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Load API tokens for our 3rd party APIs

In [21]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [22]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

In [25]:
for key in configs.keys():
    print(key)
    for k,v in configs[key].items():
        print(k,': ',v)
    print()

lead_agents
lead_data_agent :  {'role': 'Lead Data Specialist\n', 'goal': 'Collect and analyze personal and company-level data for each lead.\n', 'backstory': 'You are a data specialist with a keen eye for detail, responsible for gathering and analyzing both personal and company data to determine the potential of each lead.\n', 'verbose': True, 'allow_delegation': False}
cultural_fit_agent :  {'role': 'Cultural Fit Analyst\n', 'goal': "Assess the cultural alignment between the lead's company and our organization, and evaluate the overall fit.\n", 'backstory': 'You specialize in understanding company cultures and values, ensuring that partnerships are not only strategic but also culturally aligned for long-term success.\n', 'verbose': True, 'allow_delegation': False}
scoring_validation_agent :  {'role': 'Lead Scorer and Validator\n', 'goal': 'Aggregate data, assign a lead score based on predefined criteria, and validate the scoring accuracy.\n', 'backstory': 'You are an expert in evalua

## Create Pydantic Models for Structured Output

In [23]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [24]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [26]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

2025-01-16 07:09:29,753 - 139885878500224 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Email Engagement Crew

In [27]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2025-01-16 07:09:37,172 - 139885878500224 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [36]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "Ikram khan",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "ikram@clearbit.com",
                    "use_case": "Taking Engineering team on a trip to paris"
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [37]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [38]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [39]:
emails = await flow.kickoff()

2025-01-16 07:12:34,990 - 139885878500224 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
    - Company Name: Eiravox
    - Product: Travel Agency
    - ICP: Traveler looking for trust worthy travel agenc



# Agent: Lead Data Specialist
## Thought: Thought: Since accessing Bloomberg was unsuccessful, I will gather the necessary company details from other profiles.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company size revenue industry market presence\"}"
## Tool Output: 

Search results: Title: Clearbit - Market Share, Competitor Insights in Market Research
Link: https://www.6sense.com/tech/market-research/clearbit-market-share
Snippet: The majority of Clearbit's customers for the market-research category fall in the company size of 20 - 49 employees (1,462 companies). You can view a ...
---
Title: Leverage 100+ business data attributes - Clearbit
Link: https://clearbit.com/attributes
Snippet: Enrich your CRM and database, manage leads efficiently, and enable marketing personalization with over 100 business data points.
---
Title: Clearbit - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/clearbit
Snippet: Clearb



# Agent: Lead Data Specialist
## Final Answer: 
**Comprehensive Data Report**:

**Personal Information:**
- **Name**: Ikram Khan
- **Job Title**: Director of Engineering
- **Role Relevance**: 8
- **Professional Background**: Specific details not available but likely extensive experience in engineering leadership. 

**Company Information:**
- **Company Name**: Clearbit
- **Industry**: Business/Productivity Software
- **Company Size**: 79-116 employees
- **Revenue**: $17.8 - $41.6 million annually
- **Market Presence**: Strong presence in B2B marketing intelligence.


# Agent: Cultural Fit Analyst
## Task: Assess the cultural alignment between the lead's company and our organization by considering the following:
  - Cultural Values: Analyze the company's publicly stated values and internal culture (e.g., innovation, sustainability, employee engagement).
  - Strategic Alignment: Evaluate how well the company's goals and mission align with our organization's strategic objectives.
  - Qua



# Agent: Cultural Fit Analyst
## Thought: Thought: I need to gather more information regarding Clearbit's mission, vision, and company culture from additional sources to ensure a thorough understanding for assessing the cultural alignment.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://canvasbusinessmodel.com/blogs/mission/clearbit-mission\"}"
## Tool Output: 
Mission, Vision & Core Values of Clearbit – CANVAS, SWOT, PESTEL & BCG Matrix Editable Templates for Startups
Skip to content
Framework Templates
Blogs
Search
Cart
0
/
$0.00
 EN
EN
ES
FR
PT
Mission, Vision & Core Values of ClearbitOctober 2, 2024
CLEARBIT BUNDLE
Get Full Bundle:
Porter's Five Forces
$15
$10
BCG Matrix
$15
$10
Canvas
$15
$10
Marketing Mix
$15
$10
PESTLE Analysis
$15
$10
SWOT Analysis
$15
$10
TOTAL: ADD SELECTED TO CART 
At Clearbit, our mission drives every decision we make, shaping our vision for the future and guiding our core values. We are committed to providing innovative s



# Agent: Cultural Fit Analyst
## Final Answer: 
- **Cultural Fit Score**: 8/10  
- **Supporting Analysis and Comments**: Both Eiravox and Clearbit share core values in trust, innovation, and a focus on customer needs. Clearbit’s mission of data empowerment aligns well with Eiravox’s technology-driven travel services. While the differing sectors (business software vs. travel) present unique challenges, the underlying cultural principles create a significant potential for strategic alignment and long-term partnership success.


# Agent: Lead Scorer and Validator
## Task: Aggregate the collected data and perform the following steps: - Score Calculation: Based on predefined criteria, calculate a final lead score (0-100). Consider factors such as:
  - Role Relevance
  - Company Size
  - Market Presence
  - Cultural Fit
- Scoring Criteria Documentation: List the criteria used to determine the score. - Validation: Review the collected data and the calculated score for consistency and accura

2025-01-16 07:13:25,450 - 139885878500224 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Email Content Writer
## Task: Craft a highly personalized email using the lead's name, job title, company information, and any relevant personal or company achievements. The email should speak directly to the lead's interests and the needs of their company. This is not as cold outreach as it is a follow up to a lead form, so keep it short and to the point. Don't use any salutations or closing remarks, nor too complex sentences.
Our Company and Product: - Company Name: Eiravox - Product: Travel Agency - ICP: Traveler looking for trust worthy travel agency. - Pitch: We are a trust worthy agency to plan your trip with ai powerd chatbot.
Use the following information: Personal Info: {'name': 'Ikram Khan', 'job_title': 'Director of Engineering', 'role_relevance': 10, 'professional_background': None} Company Info: {'company_name': 'Clearbit', 'industry': 'B2B Data Provider', 'company_size': 49, 'revenue': None, 'market_presence': 10} Lead Score: {'score': 80, 'scoring_criteria': ['R

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [40]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0096


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,63762,58117,5645,30


In [41]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,1730,1453,277,3


## Inspecting Results

In [42]:
scores = flow.state["score_crews_results"]

In [43]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,Ikram Khan
Job Title,Director of Engineering
Role Relevance,10
Professional Background,None
Company Name,Clearbit
Industry,B2B Data Provider
Company Size,49
Revenue,None
Market Presence,10
Lead Score,80


## Results

In [44]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Discover how Eiravox can transform your travel planning with our AI-powered
chatbot. Say goodbye to stress and hello to seamless bookings.   Ready to
simplify your next trip? Click here to schedule a meeting with us: [Schedule a
Meeting](#)  Experience the difference of a trustworthy travel agency today.
Don't miss out on securing your perfect travel experience—act now!


## How Complex Can it Get?

In [45]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "Ikram khan",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "ikram@clearbit.com",
          "use_case": "Taking Engineering team on a trip to paris."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [46]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [47]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)